In [ ]:
import os
import json
import sqlite3
import pandas as pd
from typing import List, Dict, Any, Tuple, Optional

from formating import convert_json_to_insert_sql, convert_json_to_update_sql
from crud import insert_records, delete_records, fetch_records, get_tables

In [ ]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
assets_dir = os.path.join(parent_dir, '_assets')
db_dir = os.path.join(parent_dir, 'fetusapp')
db_filename = 'data.sqlite'
data_filename = 'detoraki_fetusdata.json'
mapping_filename = 'column_mapping.json'

db_path = os.path.join(db_dir, db_filename)
json_file = os.path.join(assets_dir, data_filename)
mapping_file = os.path.join(current_dir, mapping_filename)

Load the source data

In [ ]:
with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

Load the column mapping from the JSON file

In [ ]:
with open(mapping_file, 'r', encoding='utf-8') as f:
        column_mapping = json.load(f)

### Setup admin in Users (INSERT)

The script should already be there in _assets

In [ ]:
target_tablename = 'users'
insert_records(target_tablename, db_path)


In [ ]:
fetch_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

---------------------

### Migrate Patients (INSERT)

Includes manual step to treat patients missing last_name

In [ ]:
source_tablename = 's_org_ext'
target_tablename = 'patients'
data_index = 3

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

#### Data Cleansing:
* Identify records with null last name and either 
    * convert the null to the string 'null' in the generated script or
    * remove the record altogether if it doesn't have any useful payload

In [ ]:
patient_data = json_data[data_index]['data']
null_lastname_ids = [item['s_org_extID'] for item in patient_data if item['lastName'] == ""]
print(null_lastname_ids)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

---------------------

### Migrate MedicalHistory (INSERT)

In [ ]:
source_tablename = 'tblmedhistory'
target_tablename = 'history_medical'
data_index = 10

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

------------------

### Migrate Short Gynaecological History data (UPDATE)

In [ ]:
source_tablename = 'tblgynhistoryshort'
target_tablename = 'history_medical'
data_index = 7

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

---------

### Migrate Pathologies (UPDATE)

In [ ]:
source_tablename = 'tblpathologies'
target_tablename = 'history_medical'
data_index = 13

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

----------

### Migrate ObstetricsHistory (INSERT)

In [ ]:
source_tablename = 'tblobstetrichistory'
target_tablename = 'history_obstetrics'
data_index = 11 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

### tblobstetrichistory_x NOT TO BE MIGRATED

### Migrate Short Pregnancies History data (INSERT)

In [ ]:
source_tablename = 'tblpregnancyhistory'
target_tablename = 'pregnancy_history'
data_index = 16 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

### Migrate Short Pregnancies History X data (INSERT)

In [ ]:
source_tablename = 'tblpregnancyhistory_x'
target_tablename = 'pregnancy_history_x'
data_index = 17 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)
fetch_records(target_tablename, db_path)
# delete_records(target_tablename, db_path)

Delete records with null TER

In [ ]:


conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
DELETE FROM pregnancy_history_x
WHERE pregnancy_id IN (SELECT id FROM pregnancy_history WHERE ter IS NULL)
""")
cursor.execute("""
DELETE FROM pregnancy_history
WHERE ter IS NULL
""")
conn.commit()

conn.close()

### Migrate Gyn History X data (INSERT)

DELETE 5,8 and 10 records

In [ ]:
source_tablename = 'tblgynhistory'
target_tablename = 'gyn_history'
data_index = 6 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)
insert_records(target_tablename, db_path)
fetch_records(target_tablename, db_path)

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("""
    UPDATE gyn_history
    SET comments = TRIM(REPLACE(REPLACE(comments, '\n\n', '\n'), '\r\n\r\n', '\n'))
    WHERE comments IS NOT NULL
""")
conn.commit()
conn.close()

## Update next_suggested_appointment date

In [ ]:
from fetusapp.chatai.note_to_app_service import analyze_doctor_note
from loguru import logger

In [26]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
               SELECT patient_id
                    , date_of_visit
                    , comments
               FROM gyn_history
               WHERE date_of_visit >= date('2024-06-13', '-1 year')
               AND is_active = 1
               AND created_by = 1
               GROUP BY patient_id
               ORDER BY 2 DESC;
               """)
visits = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

visits_df = pd.DataFrame(visits, columns=columns)
visits_df

,patient_id,date_of_visit,comments
0,1349,2024-05-28,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη:\n-υ...
1,608,2024-05-09,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: \n-...
2,671,2024-04-24,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: \n-...
3,255,2024-04-24,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη:\n-υ...
4,880,2024-04-23,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη:\n-α...
...,...,...,...
392,1042,2023-06-13,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: αρν...
393,934,2023-06-13,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: αρν...
394,852,2023-06-13,-της πήρα επαναληπτικό τεστ Παπ και το πήγα στ...
395,796,2023-06-13,-καυτηριασμός σπίλων\n-μου πλήρωσε 50 ευρώ και...


In [25]:
for i, row in visits_df.head(20).iterrows():

    logger.info(f"Analyzing comment #{i}:\n{row['comments']}...")
    doctor_note = row['comments']
    patient_id = row['patient_id']
    analyze_doctor_note(doctor_note, patient_id, current_appointment_date=None)

2025-10-17 11:41:15.790 | INFO     | __main__:<module>:3 - Analyzing comment #0:
-της πήρα τεστ Παπ και το πήγα στον Λαΐνη:
-υπέρηχος εγο:
-4 μήνες μετά τη διακοπή του θηλασμού, θα κάνουμε υπέρηχο μαστών
-αν θέλει σπιράλ, θα της πω 200 ευρώ στο ΙΑΣΩ λόγω ΠΚΤ
-μου πλήρωσε 100 ευρώ...




> Entering new AgentExecutor chain...


2025-10-17 11:41:16.375 | INFO     | __main__:<module>:3 - Analyzing comment #1:
-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: 
-υπέρηχος εγο: ΠΚΚ,
-της έγραψα να κάνει μαστογραφία και υπέρηχο μαστών και θα μου τα προσκομίσει:
-μου πλήρωσε 80 ευρώ για όλα μαζί...


No appointment suggestion found.

> Finished chain.


> Entering new AgentExecutor chain...


2025-10-17 11:41:17.021 | INFO     | __main__:<module>:3 - Analyzing comment #2:
-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: 
-υπέρηχος εγο: ενδομήτριο. Εξαρτήματα δεν έχει
-θα κάνει μαστογραφία και υπέρηχο μαστών και θα μου τα προσκομίσει: 
-μου πλήρωσε 70 ευρώ για όλα μαζί...


...

No appointment suggestion found.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `calculate_appointment_date` with `{'reference_date': '2024-04-15', 'timeframe': '1 year'}`


2025-04-15

2025-10-17 11:41:30.285 | INFO     | __main__:<module>:3 - Analyzing comment #18:
-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: αρνητικό για κακοήθεια. Επανάληψη σε ένα έτος
-υπέρηχος εγο: σε ευθειωρία, χωρίς παθολογικά ευρήματα
-να κάνει μαστογραφία και υπέρηχο μαστών:
-μου πλήρωσε 100 ευρώ...



Invoking: `fetch_next_suggested_app_date` with `{'patient_id': '1121'}`




> Entering new AgentExecutor chain...

Invoking: `calculate_appointment_date` with `{'reference_date': '2024-04-15', 'timeframe': '1 year'}`


2025-04-15

2025-10-17 11:41:32.235 | INFO     | __main__:<module>:3 - Analyzing comment #19:
-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: αρνητικό για κακοήθεια. Επανάληψη σε ένα έτος
-υπέρηχος εγο: ΠΚΚ, check IUD (το είχαμε βάλει Ιούνιο 2020). Όλα κφ
-της έγραψα να κάνει μαστογραφία και υπέρηχο μαστών και θα μου τα προσκομίσει: υπέρηχος (Απρίλιος 2024): δύο απλές κύστεις του μισού εκατοστού. Εκκρεμεί μαστογραφία
-μου πλήρωσε 80 ευρώ...



Invoking: `fetch_next_suggested_app_date` with `{'patient_id': '1067'}`




> Entering new AgentExecutor chain...

Invoking: `calculate_appointment_date` with `{'reference_date': '2024-04-15', 'timeframe': '1 year'}`


2025-04-15
Invoking: `fetch_next_suggested_app_date` with `{'patient_id': '256'}`




### Migrate Medical Exams (INSERT)

In [ ]:
source_tablename = 'tblmedexam'
target_tablename = 'medical_exams'
data_index = 8 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
# delete_records(target_tablename, db_path)
insert_records(target_tablename, db_path)
fetch_records(target_tablename, db_path)

### ... to be continued

In [ ]:
get_tables(db_path)

--------------